# El Modelo Relacional

Luego de describir las operaciones necesarias para generar las tablas necesarias, estamos listos para avanzar con el proceso. Algunas líneas no serán necesarias al escribir el pipeline de Bamboo y están aquí sólo por motivos pedagógicos. Comenzamos cargando el archivo tidy que creamos en la sección anterior.

In [54]:
import pandas as pd
import csv

In [55]:
df = pd.read_csv("data_temp/tidy_file.csv")
df.head()

,region,data_origin,year,variable,response,percentage
0,Amazonas,INEI,2007,Acceso a TIC,No accede,0.798952
1,Ancash,INEI,2007,Acceso a TIC,No accede,0.560716
2,Apurimac,INEI,2007,Acceso a TIC,No accede,0.818601
3,Arequipa,INEI,2007,Acceso a TIC,No accede,0.392133
4,Ayacucho,INEI,2007,Acceso a TIC,No accede,0.785555


### Tabla de Dimensión: Region

Para tener una idea de cómo se verán nuestras tablas en la base de datos, iremos guardando archivos `.csv` de cada una en la carpeta `data_output`. Esto puede o no ser necesario en el pipeline de Bamboo, pero tener los archivos es útil para generar diccionarios de mapeo posteriormente.

In [56]:
# Extraemos los valores únicos de la columna "region" y los convertimos a una lista:
region_list = list(df["region"].unique())

# Creamos un nuevo DataFrame ingresando un diccionario con lo que necesitamos:
df_region = pd.DataFrame({"region_id": list(range(len(region_list))), "region_name": sorted(region_list)})

# Guardamos el nuevo DataFrame
df_region.to_csv("data_output/tic_dim_region.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

df_region.head()

,region_id,region_name
0,0,Amazonas
1,1,Ancash
2,2,Apurimac
3,3,Arequipa
4,4,Ayacucho


### Tabla de Dimensión: Variable

Este proceso puede ser más complejo, ya que hay valores repetidos en **response**, para solucionar este problema, crearemos una columna nueva usando **variable** y **response** combinadas, usamos un carácter inusual ("|") para luego poder separarlas otra vez.

In [57]:
# Extraemos las columnas necesarias, hacemos una copia de ellas para evitar "SettingWithCopyWarning":
df_var = df[["variable", "response"]].copy()

# Las combinamos usando el carácter | y nos quedamos sólo con la nueva columna "combined":
df_var["combined"] = df_var["variable"] + "|" + df_var["response"]
df_var = df_var[["combined"]]

# Obtenemos sólo los valores únicos de "combined", eliminando los duplicados 
# y reseteamos el índice del DataFrame, para que empiece de 0 otra vez.
df_var = df_var.drop_duplicates().reset_index(drop=True)

# Creamos una columna para los ids usando los índices nuevos, y separamos la columna "combined".
df_var["response_id"] = df_var.index
df_var["variable_name"] = df_var["combined"].str.split("|").str[0]
df_var["response_name"] = df_var["combined"].str.split("|").str[1]

# Reordenamos, podríamos dejar fuera la columna "combined" para eliminarla, 
# pero es mejor conservarla para mapear en la Fact Table.
df_var = df_var[["response_id", "variable_name", "response_name", "combined"]]

# Guardamos la tabla:
df_var.to_csv("data_output/tic_dim_variable.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

df_var

,response_id,variable_name,response_name,combined
0,0,Acceso a TIC,No accede,Acceso a TIC|No accede
1,1,Acceso a TIC,Accede,Acceso a TIC|Accede
2,2,Acceso a Internet,Accede,Acceso a Internet|Accede
3,3,Acceso a Internet,No accede,Acceso a Internet|No accede
4,4,Acceso a TV Cable,Accede,Acceso a TV Cable|Accede
5,5,Acceso a TV Cable,No accede,Acceso a TV Cable|No accede
6,6,Tipo de Teléfono,Teléfono fijo,Tipo de Teléfono|Teléfono fijo
7,7,Tipo de Teléfono,Teléfono celular,Tipo de Teléfono|Teléfono celular
8,8,Empleó Equipos Informáticos,Empleó,Empleó Equipos Informáticos|Empleó
9,9,Empleó Equipos Informáticos,No empleó,Empleó Equipos Informáticos|No empleó


### `.map()` y `.replace()`

Para generar la fact table realizaremos varios "mapeos". Las columnas en pandas poseen dos métodos especiales: `.map()` y `.replace()`. Estas funciones son capaces de modificar los valores de la columna, de acuerdo a una relación inyectiva que se defina mediante un diccionario. 

Imagina que tenemos la columna con valores `["Sí", "No", "No", "Sí", "Tal Vez", "No", "No", "Si"]` y defino un diccionario como `my_dict = {"Sí": 1, "No": 0}`, la función `.map(my_dict)` es muy estricta, hará todos los cambios posibles, pero si no encuentra un valor dentro del mapeo, retornará un valor nulo (`NaN`), el resultado será: `[1,0,0,1,NaN,0,0,NaN]` ("Tal Vez" no está en el diccionario, ni tampoco "Si" sin tilde). La diferencia con la función `.replace(my_dict)` es que dejará los valores intactos cuando no encuentre un mapeo adecuado en el diccionario, el resultado será: `[1,0,0,1,"Tal Vez",0,0,"Si"]`. 

Como preferencia personal, siempre uso `.map()` porque es sencillo revisar en el DataFrame si hay valores nulos e incluso ver dónde está fallando el diccionario para agregar lo necesario y que quede perfecto, de esa forma ejercemos mayor control en la limpieza. `.replace()` dejará pasar todos los casos no previstos, si hubiésemos tenido un caso como `["Sí", "Sí", 0, 0, "No"]` el resultado con **replace** sería `[1,1,0,0,0]` pero los 0 no fueron mapeados correctamente, tal vez significan otra cosa, **map** retornaría `[1,1,NaN,NaN,0]` sin dejar pasar los 0 desapercibidos.


### `zip()`

La función `zip` en Python es capaz de generar una lista de tuplas a partir de dos listas, por ejemplo `A = [1,2,3]` y `B = [4,5,6]`, la función `zip(A,B)` retorna `[(1,4),(2,5),(3,6)]` y si por ejemplo quisiera una lista con las sumas de los pares, podría generar todo con una comprensión de lista `S = [a+b for (a,b) in zip([1,2,3],[4,5,6])]`. Este método se puede utilizar para generar diccionarios de mapeo, iteramos sobre pares de `key:value` a partir de dos columnas en un DataFrame.


### Fact Table

In [58]:
# Creamos el diccionario de mapeo para regiones y realizamos el mapeo,
# Como algunos comandos son muy largos, usaré nombres de DataFrame abreviados:
df_reg = pd.read_csv("data_output/tic_dim_region.csv")
region_map = {k:v for (k,v) in zip(df_reg["region_name"], df_reg["region_id"])}
df["region_id"] = df["region"].map(region_map)

# Creamos el diccionario de mapeo para orígenes de los datos manualmente y lo utilizamos:
origin_map = {"INEI": 1, "ENE": 0}
df["data_origin_id"] = df["data_origin"].map(origin_map)

# Creamos el diccionario de mapeo de variable usando la columna "combined", por lo tanto
# tendremos que llevar las columnas "variable/response" a "combined" para mapear en la fact table.
df_var = pd.read_csv("data_output/tic_dim_variable.csv")
variable_map = {k:v for (k,v) in zip(df_var["combined"], df_var["response_id"])}
df["combined"] = df["variable"] + "|" + df["response"]
df["response_id"] = df["combined"].map(variable_map)

# Reordenamos las columnas, dejando fuera las que queremos eliminar:
df = df[["region_id", "data_origin_id", "response_id", "year", "percentage"]]

# Guardamos la Fact Table
df.to_csv("data_output/tic_fact.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

df.head()

,region_id,data_origin_id,response_id,year,percentage
0,0,1,0,2007,0.798952
1,1,1,0,2007,0.560716
2,2,1,0,2007,0.818601
3,3,1,0,2007,0.392133
4,4,1,0,2007,0.785555


Es importante verificar que no tenemos valores nulos en nuestra Fact Table en columnas importantes.

In [59]:
df.isnull().any()

region_id         False
data_origin_id    False
response_id       False
year              False
percentage        False
dtype: bool

Y ahora que ya tenemos el código necesario para crear cada tabla podemos escribir el Pipeline de Bamboo en la siguiente sección.